In [1]:
""":file: MEV_telephone_cross_var.py

Version for Biogeme 3.2.13

:author: Michel Bierlaire
:date: Fri Apr 28 10:19:38 2023

"""

':file: MEV_telephone_cross_var.py\n\nVersion for Biogeme 3.2.13\n\n:author: Michel Bierlaire\n:date: Fri Apr 28 10:19:38 2023\n\n'

In [2]:
import biogeme.results as res
from biogeme.exceptions import BiogemeError
import biogeme.biogeme as bio
from biogeme.expressions import Beta
from biogeme.models import logcnl
from biogeme.nests import OneNestForCrossNestedLogit, NestsForCrossNestedLogit

In [3]:
from telephone_data import (
    database,
    choice,
    avail1,
    avail2,
    avail3,
    avail4,
    avail5,
    logcostBM,
    logcostSM,
    logcostLF,
    logcostEF,
    logcostMF,
)

Parameters to be estimated
Arguments:
  1  Name for report. Typically, the same as the variable
  2  Starting value
  3  Lower bound
  4  Upper bound
  5  0: estimate the parameter, 1: keep it fixed

In [4]:
ASC_BM = Beta('ASC_BM', 0, None, None, 0)
ASC_EF = Beta('ASC_EF', 0, None, None, 0)
ASC_LF = Beta('ASC_LF', 0, None, None, 0)
ASC_MF = Beta('ASC_MF', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

Nest parameters

In [5]:
N_FLAT = Beta('N_FLAT', 1, None, None, 0)
N_MEAS = Beta('N_MEAS', 1, None, None, 0)

In [6]:
a_FLAT_LF = Beta('a_FLAT_LF', 0.5, 0, 1, 0)
a_MEAS_LF = 1 - a_FLAT_LF

Utilities

In [7]:
V_BM = ASC_BM + B_COST * logcostBM
V_SM = B_COST * logcostSM
V_LF = ASC_LF + B_COST * logcostLF
V_EF = ASC_EF + B_COST * logcostEF
V_MF = ASC_MF + B_COST * logcostMF

In [8]:
V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

Definitions of nests

In [9]:
alpha_N_FLAT = {1: 0, 2: 0, 3: a_FLAT_LF, 4: 1, 5: 1}
alpha_N_MEAS = {1: 1, 2: 1, 3: a_MEAS_LF, 4: 0, 5: 0}

In [10]:
nest_N_FLAT = OneNestForCrossNestedLogit(
    nest_param=N_FLAT, dict_of_alpha=alpha_N_FLAT, name='flat'
)
nest_N_MEAS = OneNestForCrossNestedLogit(
    nest_param=N_MEAS, dict_of_alpha=alpha_N_MEAS, name='measured'
)

In [11]:
nests = NestsForCrossNestedLogit(
    choice_set=list(V), tuple_of_nests=(nest_N_FLAT, nest_N_MEAS)
)

CNL with fixed alphas

In [12]:
logprob = logcnl(V, avail, nests, choice)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'MEV_telephone_cross_var'
the_biogeme.calculateNullLoglikelihood(avail)
results = the_biogeme.estimate()

Get the results in a pandas table

In [13]:
pandasResults = results.getEstimatedParameters()
print(pandasResults)

              Value  Active bound  Rob. Std err   Rob. t-test  Rob. p-value
ASC_BM    -0.567638           0.0  1.656976e-01 -3.425748e+00  6.131083e-04
ASC_EF    -1.625943           0.0  3.293393e+00 -4.936984e-01  6.215192e-01
ASC_LF     0.570260           0.0  3.504795e-01  1.627085e+00  1.037190e-01
ASC_MF     0.214317           0.0  1.442975e+00  1.485242e-01  8.819291e-01
B_COST    -2.143455           0.0  2.818413e-01 -7.605183e+00  2.842171e-14
N_FLAT     0.338454           0.0  1.745268e-01  1.939270e+00  5.246853e-02
N_MEAS     1.414144           0.0  3.707028e-01  3.814766e+00  1.363119e-04
a_FLAT_LF  1.000000           1.0  1.113720e-10  8.978921e+09  0.000000e+00


In [14]:
pandasResults

,Value,Active bound,Rob. Std err,Rob. t-test,Rob. p-value
ASC_BM,-0.567638,0.0,1.656976e-01,-3.425748e+00,6.131083e-04
ASC_EF,-1.625943,0.0,3.293393e+00,-4.936984e-01,6.215192e-01
ASC_LF,0.570260,0.0,3.504795e-01,1.627085e+00,1.037190e-01
ASC_MF,0.214317,0.0,1.442975e+00,1.485242e-01,8.819291e-01
B_COST,-2.143455,0.0,2.818413e-01,-7.605183e+00,2.842171e-14
N_FLAT,0.338454,0.0,1.745268e-01,1.939270e+00,5.246853e-02
N_MEAS,1.414144,0.0,3.707028e-01,3.814766e+00,1.363119e-04
a_FLAT_LF,1.000000,1.0,1.113720e-10,8.978921e+09,0.000000e+00


In [15]:
print(f'Nbr of observations: {database.getNumberOfObservations()}')
print(f'LL(0) =    {results.data.initLogLike:.3f}')
print(f'LL(beta) = {results.data.logLike:.3f}')
print(f'AIC = {results.data.akaike:.1f}')
print(f'Output file: {results.data.htmlFileName}')

Nbr of observations: 434
LL(0) =    -477.086
LL(beta) = -477.086
AIC = 970.2
Output file: MEV_telephone_cross_var~00.html


Likelihood ratio test
Compare with the nested logit model

In [16]:
try:
    nested_results = res.bioResults(pickleFile='MEV_telephone_nested.pickle')
except BiogemeError:
    print('To perform the test, run first the script MEV_telephone_nested.py')

ll_nested = nested_results.data.logLike
aic_nested = nested_results.data.akaike
ll_cross_var = results.data.logLike
aic_cross_var = results.data.akaike

print(
    f'LL nested:  {ll_nested:.3f}  '
    f'AIC: {aic_nested:.3f}  '
    f'Parameters: {nested_results.data.nparam}'
)
print(
    f'LL CNL (var): {ll_cross_var:.3f}  '
    f'AIC: {aic_cross_var:.3f}  '
    f'Parameters: {results.data.nparam}'
)

lr_test = results.likelihood_ratio_test(nested_results)
print(f'Likelihood ratio: {lr_test.statistic:.3f}')
print(f'Test threshold: {lr_test.threshold:.3f}')
print(f'Test diagnostic: {lr_test.message}')

LL nested:  -473.219  AIC: 960.439  Parameters: 7
LL CNL (var): -477.086  AIC: 970.172  Parameters: 8


BiogemeError: The unrestricted model (-477.0858410978324, 8) has a lower log likelihood than the restricted one (-473.21933315985905, 7)